# 1. Introdução

In [ ]:
#Importar bibliotecas
import pandas as pd
import numpy as np
from scipy import stats

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.preprocessing import StandardScaler

In [ ]:
#Carregar dados
base = pd.read_csv('base_final.csv')
base.head()

# 2. Explorar Dados

In [ ]:
#Separar atributos em categóricas e numéricas
colunas_categoricas = [col for col, idx in zip(base.dtypes.index, base.dtypes) if idx == 'object']
colunas_numericas = [col for col, idx in zip(base.dtypes.index, base.dtypes) if idx != 'object']

colunas_categoricas = [col for col in colunas_categoricas if 'data' not in col]
colunas_numericas.remove('id_usuario')

## 2.1. Dados Categóricos

In [ ]:
base['grupo_transacao_1'].value_counts()

In [ ]:
base_grafico = base.groupby(by = 'grupo_transacao_2').agg({'id_usuario': 'count'})
fig = go.Figure([go.Bar(x = base_grafico.index, y = base_grafico['id_usuario'])])
fig.update_layout(title = 'Distribuição do Atributo grupo_transacao_1', template = 'plotly_white')
fig.show()

In [ ]:
#grupo_transacao_1 e grupo_transacao_2
fig = make_subplots(rows = 1, cols = 2, subplot_titles = ('Primeira Transação', 'Segunda Transação'), shared_yaxes = True)

for i, col in enumerate(['grupo_transacao_1', 'grupo_transacao_2']):
    base_grafico = base.groupby(by = col).agg({'id_usuario': 'count'})
    fig.append_trace(go.Bar(x = base_grafico.index, y = base_grafico['id_usuario'], name = col), row = 1, col = i+1)

fig.update_layout(title = 'Distribuição de Grupo Transação', template = 'plotly_white', showlegend = False)
fig.show()

A primeira transação, de forma geral, é um cashin, ou em outras palavras, uma inclusão de fundos na conta. Por algum motivo, temos ainda uma pequena parcela de primeiras transações sendo cashouts, ou gastos/compras. Conhecendo os dados, esses casos provavelmente são resultado de algum problema no processamento da finalização da transação inicial de cashin. Antes de decidir que medida tomar em relação aos dados, vamos investigar como eles se relacionam com nosso target.

Para a segunda transação, temos uma concentração em cashout, ainda que não tão grande quanto o que acontece na primeira.

In [ ]:
#tipo_transacao_anonimo_1
base_grafico = base['tipo_transacao_anonimo_1'].value_counts().sort_values()
fig = go.Figure([go.Bar(y = base_grafico.index, x = base_grafico.values, orientation = 'h')])

fig.update_layout(title = 'Principais Métodos de Primeira Transação', template = 'plotly_white')
fig.show()

O que vemos é uma concentração da primeira transação em 2 métodos de cashin. Isso nos dá a possibilidade de agrupar as demais categorias em um único super-grupo.

In [ ]:
#tipo_transacao_anonimo_2
base_grafico = base['tipo_transacao_anonimo_2'].value_counts().sort_values()
fig = go.Figure([go.Bar(y = base_grafico.index, x = base_grafico.values, orientation = 'h')])

fig.update_layout(title = 'Principais Métodos de Segunda Transação', template = 'plotly_white')
fig.show()

Ainda que tenhamos certa concentração em 3 métodos de transação, a distribuição é bem melhor do que para a primeira transação.

In [ ]:
#estado_anonimo
base_grafico = base['estado_anonimo'].value_counts().sort_values()
fig = go.Figure([go.Bar(y = base_grafico.index, x = base_grafico.values, orientation = 'h')])

fig.update_layout(title = 'Distribuição dos Usuários Pelos Estados do País', template = 'plotly_white')
fig.show()

A concentração de grande parte dos usuários em poucas categorias também acontece aqui. No entanto, um agrupamento de categorias tem que ser avaliado com muito cuidado, especialmente pois temos muitas categorias que estão mascarados para efeito de segurança.

In [ ]:
#faixa_idade
base_grafico = base['faixa_idade'].value_counts().sort_index(ascending = False)
fig = go.Figure([go.Bar(y = base_grafico.index, x = base_grafico.values, orientation = 'h')])

fig.update_layout(title = 'Distribuição Por Idade do Usuários', template = 'plotly_white')
fig.show()

A gráfico de idade, considerando que as faixas criadas apresentam idades em uma sequência lógica, mostram uma distribuição bem equilibrada, sem uma concentração muito grande em qualquer segmento. 

In [ ]:
#genero
base_grafico = base['genero'].value_counts().sort_values()
fig = go.Figure([go.Bar(x = base_grafico.index, y = base_grafico.values)])

fig.update_layout(title = 'Distribuição dos Usuários Por Gênero', template = 'plotly_white')
fig.show()

Quanto à distribuição por gênero, não temos nenhuma categoria com uma concentração alta demais.

## 2.2. Dados Numéricos

In [ ]:
fig = make_subplots(rows = 1, cols = 2)
fig.append_trace(go.Histogram(x = base['valor_1'], nbinsx = 100), row = 1, col = 1)
fig.append_trace(go.Histogram(x = base['valor_2'], nbinsx = 150), row = 1, col = 2)

fig.update_yaxes(title = 'Usuários', showgrid = False, range = [0, 150000], tickformat = '.,', ticksuffix = '   ',
                 row = 1, col = 1)
fig.update_yaxes(showgrid = False, range = [0, 60000], tickformat = '###,###', ticksuffix = '   ', 
                 row = 1, col = 2)
#fig.update_xaxes(title = 'Valor 1', range = [0, 600], row = 1, col = 1)
#fig.update_xaxes(title = 'Valor 2', range = [0, 600], row = 1, col = 2)
fig.update_layout(template = 'plotly_white', showlegend = False, xaxis = {'range': [0, 2000]}, 
                  font = {'family': 'Arial', 'color': '#7c7c7d', 'size': 16})
fig.show()

In [ ]:
fig = make_subplots(rows = 1, cols = 2)
fig.append_trace(go.Histogram(x = stats.boxcox(base['valor_1'])[0], nbinsx = 250), row = 1, col = 1)
fig.append_trace(go.Histogram(x = stats.boxcox(base['valor_2'])[0], nbinsx = 250), row = 1, col = 2)

fig.update_yaxes(title = 'Usuários', showgrid = False, tickformat = '###.###', ticksuffix = '   ', row = 1, col = 1)
fig.update_yaxes(showgrid = False, tickformat = '###.###', ticksuffix = '   ', row = 1, col = 2)
fig.update_xaxes(title = 'Valor 1', row = 1, col = 1)
fig.update_xaxes(title = 'Valor 2', row = 1, col = 2)
fig.update_layout(template = 'plotly_white', showlegend = False,  
                  font = {'family': 'Arial', 'color': '#7c7c7d', 'size': 16})
fig.show()

O primeiro gráfico de distribuição dos valores mostra que temos uma grande concentração em valores baixos, com alguns valores muito altos. Temos, basicamente, campos bastante mal distribuídos. Isso pode ser visto de outra forma, comparando os valores médio e mediano para os campos.

In [ ]:
#Printar média e mediana do valor das transações
print('Valor Primeira Transação: Média = {:.2f} / Mediana = {:.2f}'.format(base['valor_1'].mean(), base['valor_1'].median())) 
print('Valor Segunda Transação: Média = {:.2f} / Mediana = {:.2f}'.format(base['valor_2'].mean(), base['valor_2'].median())) 

Com essa má distribuição dos dados, faz sentido avaliar uma remoção dos outliers, especialmente para a primeira transação

In [ ]:
#Calcular limiar para remoção de outliers
valor_1_max = base['valor_1'].mean() + 3 * base['valor_1'].std()
valor_2_max = base['valor_2'].mean() + 3 * base['valor_2'].std()

base_aux = base[(base['valor_1'] < valor_1_max) & (base['valor_2'] < valor_2_max)]
print('Seriam removidos, com os limites calculados, {:.2f}% dos registros,'.format(100 * (1 - len(base_aux) / len(base))))

In [ ]:
base = base[(base['valor_1'] < valor_1_max) & (base['valor_2'] < valor_2_max)]

In [ ]:
fig = make_subplots(rows = 1, cols = 2)
fig.append_trace(go.Histogram(x = base['dias_ate_primeira_transacao'], nbinsx = 150), row = 1, col = 1)
fig.append_trace(go.Histogram(x = base['dias_entre_transacoes'], nbinsx = 150), row = 1, col = 2)

fig.update_yaxes(title = 'Usuários', showgrid = False, tickformat = '###.###', ticksuffix = '   ', row = 1, col = 1)
fig.update_yaxes(showgrid = False, tickformat = '###.###', ticksuffix = '   ', row = 1, col = 2)
fig.update_xaxes(title = 'Dias Até Primeira Transação', row = 1, col = 1)
fig.update_xaxes(title = 'Dias Entre Transações', row = 1, col = 2)
fig.update_layout(template = 'plotly_white', showlegend = False,  
                  font = {'family': 'Arial', 'color': '#7c7c7d', 'size': 16})
fig.show()

In [ ]:
#dias_ate_primeira_transacao
fig = go.Figure([go.Histogram(x = base['dias_ate_primeira_transacao'], xbins = {'size': 1})])

fig.update_layout(title = 'Dias Entre Criação da Conta e Primeira Transação', template = 'plotly_white')
fig.show()

Temos mais uma vez um campo extremamente mal distribuído. Para resolver isso, vamos aplicar um agrupamento baseado no comportamento do produto.

In [ ]:
#Agrupar valores de dias_ate_primeira_transacao
faixas = {
    'A. 0 Dia': [-10, 0],
    'B. 1 Dia': [1, 1],
    'C. 2-7 Dias': [2, 7],
    'D. 8-30 Dias': [8, 30],
    'E. 31+ Dias': [31, 1000]}

dias_tx = []
for dias in base['dias_ate_primeira_transacao']:
    if dias == dias:
        for faixa, limites in faixas.items():
            if dias >= limites[0] and dias <= limites[1]:
                dias_tx.append(faixa)
                break
    else:
        dias_tx.append(dias)

base['faixa_dias_ate_primeira_transacao'] = dias_tx

In [ ]:
#Plotar distribuição de faixa_dias_ate_primeira_transacao
base_grafico = base['faixa_dias_ate_primeira_transacao'].value_counts().sort_index()
fig = go.Figure([go.Bar(x = base_grafico.index, y = base_grafico.values)])
fig.update_layout(title = 'Distribuição Por Dias Até Primeira Transação', template = 'plotly_white')
fig.show()

* A maior parte das primeiras transaçõesa acontece no mesmo dia que a criação conta;
* Dos casos restantes, vemos um concentração maior nos últimos dois seguimentos.

In [ ]:
#dias_entre_transacoes
print('Percentual de registros nulos: {:.2f}%'.format(100 * base['dias_entre_transacoes'].isnull().sum() / len(base)))

In [ ]:
#PLotar distribuição dos valores válidos de dias_entre_transacoes
fig = go.Figure([go.Histogram(x = base['dias_entre_transacoes'])])
fig.update_layout(title = 'Distribuição de Dias Entre Transações', template = 'plotly_white')
fig.show()

A distribuição dos valores mostra uma forte concentração para o comportamento de realizar a segunda transação logo após a primeira. Para melhorar o comportamento do atributo, vamos agrupá-lo em faixas. 

In [ ]:
#Plotar nova distribuição de dias_entre_transacoes
faixas = {
    'A. 0 Dia': [-10, 0],
    'B. 1-7 Dias': [1, 7],
    'C. 8-30 Dias': [8, 30],
    'D. 31-60 Dias': [31, 60],
    'E. 61+ Dias': [31, 1000]}

dias_tx = []
for dias in base['dias_entre_transacoes']:
    if dias == dias:
        for faixa, limites in faixas.items():
            if dias >= limites[0] and dias <= limites[1]:
                dias_tx.append(faixa)
                break
    else:
        dias_tx.append(dias)

base['faixa_dias_entre_transacoes'] = dias_tx
base_grafico = base[base['dias_entre_transacoes'].isnull() == False]['faixa_dias_entre_transacoes'].value_counts().sort_index()
fig = go.Figure([go.Bar(x = base_grafico.index, y = base_grafico.values)])
fig.update_layout(title = 'Distribuição de Dias Entre Transações', template = 'plotly_white')
fig.show()

In [ ]:
fig = make_subplots(rows = 1, cols = 2)
fig.append_trace(go.Histogram(x = base['dias_ate_primeira_transacao']), row = 1, col = 1)
fig.append_trace(go.Histogram(x = base['dias_entre_transacoes']), row = 1, col = 2)

fig.update_yaxes(title = 'Usuários', showgrid = False, tickformat = '###.###', ticksuffix = '   ', range = [0, 15000])
fig.update_xaxes(title = 'dias_ate_primeira_transacao', row = 1, col = 1)
fig.update_xaxes(title = 'dias_entre_transacoes', row = 1, col = 2)
fig.update_layout(template = 'plotly_white', showlegend = False,  
                  font = {'family': 'Arial', 'color': '#7c7c7d', 'size': 16})
fig.show()

In [ ]:
fig = make_subplots(rows = 1, cols = 2)

base_grafico = base[base['dias_ate_primeira_transacao'].isnull() == False]['faixa_dias_ate_primeira_transacao'].value_counts().sort_index()
fig.append_trace(go.Bar(x = base_grafico.index, y = base_grafico.values), row = 1, col = 1)

base_grafico = base[base['dias_entre_transacoes'].isnull() == False]['faixa_dias_entre_transacoes'].value_counts().sort_index()
fig.append_trace(go.Bar(x = base_grafico.index, y = base_grafico.values), row = 1, col = 2)

fig.update_yaxes(showgrid = False, tickformat = '###.###', ticksuffix = '   ')
fig.update_xaxes(title = 'dias_ate_primeira_transacao', row = 1, col = 1)
fig.update_xaxes(title = 'dias_entre_transacoes', row = 1, col = 2)
fig.update_layout(template = 'plotly_white', showlegend = False,  
                  font = {'family': 'Arial', 'color': '#7c7c7d', 'size': 15})
fig.show()

In [ ]:
#flag_capital_estado
base_grafico = base['flag_capital_do_estado'].value_counts()
fig = go.Figure([go.Bar(x = [str(val) for val in base_grafico.index], y = base_grafico.values)])
fig.update_layout(title = 'Distribuição da flag_capital_estado', template = 'plotly_white')
fig.show()

A distribuição da flag_capital_estado aponta para o fato que a maior parte da base de usuários não se encontra em capitais. Isso pode ser explicado pela alta capilaridade das Casas Bahia.

In [ ]:
#flag_retido
base_grafico = base['flag_retido'].value_counts()
fig = go.Figure([go.Bar(x = [str(val) for val in base_grafico.index], y = base_grafico.values)])
fig.update_layout(title = 'Distribuição da flag_retido', template = 'plotly_white')
fig.show()

Quanto ao nosso target, vemos que a maior parte dos usuários não se enquadra no cenário de sucesso, com uma proporção aproximada de 1:2 entre os dois resultados possíveis. Nesse caso, não temos um problema tão desbalanceado ao ponto de exigir a implementação de estratégias de compensação. 

# 3. Definição de retenção

In [ ]:
#Carregar e agregar dados de tempo de atividade
aux = pd.read_csv('base_target_final.csv')
aux = aux.groupby(by = 'meses_ativo').nunique()
aux.columns = ['usuarios']

aux.sort_index(ascending = False, inplace = True)
aux['usuarios_acumulado'] = aux['usuarios'].cumsum()

In [ ]:
aux

In [ ]:
#PLotar funil de atividade
fig = go.Figure([go.Bar(x = aux.index, y = aux['usuarios_acumulado'])])
fig.show()

# 4. Explorar Correlação Entre Campos

## 4.1. Campos Numéricos

In [ ]:
#Calcular correlação entre atributos
matriz_correlacao = base.corr()
matriz_correlacao.drop(columns = ['id_usuario', 'flag_retido'], inplace = True)
matriz_correlacao.drop(index = ['id_usuario', 'flag_retido'], inplace = True)

nomes = [' Valor 1 ', ' Valor 2 ', ' Dias Entre Transações ', ' Flag Capital do Estado ', ' Dias Até Primeira Transação ']
matriz_correlacao.columns = nomes
matriz_correlacao.index = nomes

fig = go.Figure([go.Heatmap(x = matriz_correlacao.index, y = matriz_correlacao.columns, 
                            z = matriz_correlacao, colorscale = 'Bluered')])
fig.update_xaxes(tickangle = 90)
fig.update_layout(height = 600, width = 600)
fig.show()

Avaliando exclusivamente os atributos numéricos entre si, vemos que apenas os campos de valor financeiro têm alguma correlação. Ainda assim, os valores não são tão altos que possam sugerir a necessidade de remover ou combinar campos.

In [ ]:
#Calcular correlação dos atributos ao target
matriz_correlacao = base.corr()['flag_retido'].sort_values()
matriz_correlacao.drop(index = 'flag_retido', inplace = True)

fig = go.Figure([go.Bar(y = matriz_correlacao.index, x = matriz_correlacao.values, orientation = 'h')])
fig.update_yaxes(showgrid = False, tickformat = '###.###', ticksuffix = '   ')
fig.update_xaxes(title = 'Coeficiente de correlação Pearson', showgrid = False, showline = True, range = [0, 0.5])
fig.update_layout(template = 'plotly_white', showlegend = False,  
                  font = {'family': 'Arial', 'color': '#7c7c7d', 'size': 15})

fig.show()

A correlação direta dos atributos numéricos com o nosso alvo é bem pequena.

## 4.2. Campos Categóricos

In [ ]:
#Definir função para gerar gráfico de distribuição do target como função de um campo categórico
def agrupar_target(base, target, coluna):
    base_grafico = base.groupby(by = coluna).agg({target: ['count', 'sum']})
    base_grafico.columns = ['total', 'sucesso']
    base_grafico['fracasso'] = 100 * (base_grafico['total'] - base_grafico['sucesso']) / base_grafico['total']
    base_grafico['sucesso'] = 100 * base_grafico['sucesso'] / base_grafico['total']
    
    fig = go.Figure()
    for resultado in ['sucesso', 'fracasso']:
        fig.add_trace(go.Bar(x= base_grafico.index, y = base_grafico[resultado], name = resultado))
    fig.update_layout(title = 'Distribuição do Target por {}'.format(coluna), template = 'plotly_white', barmode = 'stack')
    
    return fig.show()

In [ ]:
#faixa_idade
agrupar_target(base, 'flag_retido', 'faixa_idade')

In [ ]:
#grupo_transacao_1
agrupar_target(base, 'flag_retido', 'grupo_transacao_1')

In [ ]:
#grupo_transacao_2
agrupar_target(base, 'flag_retido', 'grupo_transacao_2')

In [ ]:
#tipo_transacao_anonimo_1
agrupar_target(base, 'flag_retido', 'tipo_transacao_anonimo_1')

In [ ]:
#tipo_transacao_anonimo_2
agrupar_target(base, 'flag_retido', 'tipo_transacao_anonimo_2')

In [ ]:
#estado_anonimo
agrupar_target(base, 'flag_retido', 'estado_anonimo')

In [ ]:
#estado_anonimo
agrupar_target(base, 'flag_retido', 'genero')

In [ ]:
#faixa_dias_ate_primeira_transacao
agrupar_target(base, 'flag_retido', 'faixa_dias_ate_primeira_transacao')

In [ ]:
#faixa_dias_entre_transacoes
agrupar_target(base, 'flag_retido', 'faixa_dias_entre_transacoes')

Conclusões Iniciais:

* A correlação entre os atributos numéricos é, de maneira geral, bem fraca. A única exceção é o par de campos valor_1 e valor_2;
* Quanto à correlação dos atributos numéricos com o campo-alvo, também vemos pouca força no relacionamento direto;
* Para os campos dias_ate_primeira_transacao e dias_entre_transacoes, a comparação entre o atributo original e sua versão agrupada aponta que o segundo parece ter melhor correlação com o alvo que o primeiro;
* No caso dos campos categóricos, investigamos a proporção de sucessos e fracassos para os diferentes valores categóricos. De forma geral, a maior parte dos atributos parece apresentar certo relacionamento com o alvo. Aqui, a principal exceção é o campo genero.

In [ ]:
#Remover campos dias_ate_primeira_transacao e dias_entre_transacoes
base.drop(columns = ['dias_ate_primeira_transacao', 'dias_entre_transacoes'], inplace = True, errors = 'ignore')

#Remover campos data_transacao_1, data_transacao_2 e data_criacao_conta
base.drop(columns = ['data_transacao_1', 'data_transacao_2', 'data_criacao_conta'], inplace = True, errors = 'ignore')

# 4. Pré-Processamento dos Dados

Após uma exploração dos dados e filtragem inicial de atributos, vamos partir para uma seleção de campos mais fundamentada nos métodos de predição. No entanto, antes de iniciarmos isso, vamos aplicar algumas etapas de pré-processamento nos dados. Basicamente, as duas operações que vamos realizar são:

* Padronização da escala dos campos de valor financeiro;
* Transformar atributos categóricos em campos binários.

## Normalizar Escala dos Atributos de Valor Financeiro

In [ ]:
#Aplicar scaling a valor_1 e valor_2
scaler = StandardScaler()
scaler.fit(np.array(list(base['valor_1'].values) + list(base['valor_1'].values)).reshape(-1,1))

base['valor_1_normalizado'] = scaler.transform(base['valor_1'].values.reshape(-1,1))
base['valor_2_normalizado'] = scaler.transform(base['valor_2'].values.reshape(-1,1))

In [ ]:
#Remover atributos originais
base.drop(columns = ['valor_1', 'valor_2'], inplace = True)

## Transformar Atributos Categóricos em Binários

In [ ]:
#Definir função para transformar atributos categóricos em binários
def cat_para_binario(base, coluna, remover_primeiro = False):
    base = pd.concat(
        [base, pd.get_dummies(data = base[coluna], prefix = coluna, prefix_sep = ':', drop_first = remover_primeiro)],
        axis = 1)
    base = base.drop(columns = coluna)
    return base

In [ ]:
#Transformar atributos em binários
atributos_1 = ['tipo_transacao_anonimo_1', 'tipo_transacao_anonimo_2', 'estado_anonimo', 'faixa_idade', 
               'faixa_dias_ate_primeira_transacao', 'faixa_dias_entre_transacoes']
atributos_2 = ['grupo_transacao_1', 'grupo_transacao_2', 'genero']

for atributo in atributos_1:
    base = cat_para_binario(base, atributo, False)
    
for atributo in atributos_2:
    base = cat_para_binario(base, atributo, True)

In [ ]:
#Salvar base em arquivo
base.to_csv('base_para_modelo.csv', index = False)